In [1]:
import numpy as np

In [ ]:
class Problem:
    
    def __init__(self, A, tau, rank):

        self.tau = tau
        self.rank = rank
        
        self.A = A
        sel.m = A.shape[0]
        self.n = A.shape[1]
        
        
    def projection(Y1, Y2):
        /////////
    
    def grad(self, svd, R):
        
        U = svd[0]
        V = svd[2]
        
        Ru = np.dot(R.T, U)
        Rv = np.dot(R, V)
        
        M = np.dot(U.T, Rv)
        
        Up = Rv - np.dot(U, M)
        Vp = Ru - np.dot(V, M.T)
        
        
        grad = U.dot(M.dot(V.T)) + Up.dot(V.T) + U.dot(Vp.T)
        grad_list = [U, M, V, Up, Vp]
        
        return (grad, grad_list)
    
    def vector_transport(self, X_old_svd, X_new_svd, v_list):
        U_old = X_old_svd[0]
        S_old = X_old_svd[1]
        V_old = X_old_svd[2]
    
        U_plus = X_new_svd[0]
        S_plus = X_new_svd[1]
        V_plus = X_new_svd[2]
        
        M = v_list[1]
        Up = v_list[3]
        Vp = v_list[4]
        
        Av = np.dot(V.T, V_plus)
        Au = np.dot(U.T, U_plus)
        
        B_v = np.dot(Vp.T, V_plus)
        B_u = np.dot(Up.T, U_plus)
        
        M_plus_one = np.dot(Au.T, np.dot(M, Av))
        U_plus_one = np.dot(U, np.dot(M, Av))
        V_plus_one = np.dot(V, np.dot(M.T, Au))
        
        M_plus_two = np.dot(Bu.T, Av)
        U_plus_two = np.dot(Up, Av)
        V_plus_two = np.dot(V, Bu)
        
        M_plus_three = np.dot(Au.T, Bv)
        U_plus_three = np.dot(U, Bv)
        V_plus_three = np.dot(Vp, Au)
        
        M_plus = M_plus_one + M_plus_two + M_plus_three
        Up_plus = U_plus_one + U_plus_two + U_plus_three
        Up_plus = Up_plus - np.dot(U_plus, np.dot(U_plus.T, Up_plus))
        
        Vp_plus = V_plus_one + V_plus_two + V_plus_three
        Vp_plus = Vp_plus - np.dot(V_plus, np.dot(V_plus.T, Vp_plus))
        
        transport = np.dot(U_plus, np.dot(M_plus, V_plus.T)) + np.dot(Up_plus, V_plus.T) + np.dot(U_plus, Vp_plus.T)
        transport_list = [U_plus, M_plus, V_plus, Up_plus, Vp_plus]
        return transport, transport_list
    
    def conjugate_direction(self, X_old_svd, X_new_svd, xi_old_list, xi_new_list, eta_old_list):
        
        # Transport previous gradient and direction to current tangent space:
        xi_bar, xi_bar_list = self.vector_transport(X_old_svd, X_new_svd, xi_old_list)
        eta_bar, eta_bar_list = self.vector_transport(X_old_svd, X_new_svd, eta_old_list)
        
        # Compute conjugate direction
        delta = xi - xi_bar
        top = np.dot(delta.T, xi)
        bottom = np.dot(xi_prev.T, xi_prev)
        betta = np.max(0, top/bottom)
        eta = -xi + betta*eta_bar
        
        # Renew eta_list
        U_eta_bar = eta_bar_list[0]
        M_eta_bar = eta_bar_list[1]
        V_eta_bar = eta_bar_list[2]
        Up_eta_bar = eta_bar_list[3]
        Vp_eta_bar = eta_bar_list[4]
        
        
        M_xi = xi_bar_list[1]
        Up_xi = xi_bar_list[3]
        Vp_xi = xi_bar_list[4]
        
        M_eta = -M_xi + betta*M_eta_bar
        Up_eta = -Up_xi + betta*Up_eta_bar
        Vp_eta = -Vp_xi + betta*Vp_eta_bar
        U_eta = U_eta_bar.copy()
        V_eta = V_eta_bar.copy()
        
        eta_list = [U_eta, M_eta, V_eta, Up_eta, Vp_eta]
        
        # Compute angle between conjugate direction and gradient:
        
        top = np.dot(eta.T, xi)
        bottom = np.sqrt(np.dot(eta.T, eta)*np.dot(xi.T, xi))
        alpha = top/bottom
        
        # Reset to gradient if desired:
        if alpha <= 0.1:
            eta = xi.copy()
            eta_list = xi_new_list.copy()
        
        return (eta, eta_list)
    
    def compute_initial_guess(self, X_new_svd, Rw, eta_new_list):
        
        U = X_new_svd[0]
        V = X_new_svd[2]
        M = eta_new_list[1]
        Up = eta_new_list[3]
        Vp = eta_new_list[4]
        
        Y1 = np.dot(U, M) + Up
        Y1 = np.hstack((Y1, U))
        Y2 = np.hstack((V, Vp))
        
        N = self.projection(Y1, Y2)
        top = np.dot(N.T, R)
        bottom = np.dot(N.T, N)
        t = np.trace(top)/ np.trace(bottom)
        return t
            
    
    def compute_retraction(self, X_new_svd, xi_new_list):
        
        k = self.rank
        
        Zero = np.zeros(Up.shape)
        First = np.hstack((Up, Zero))
        (Qu, Ru) = np.linalg.qr(First)
        
        Second = np.hstack((Vp, Zero))
        (Qv, Rv) = np.linalg.qr(Second)
        
        top = np.hstack((S + M, Rv.T))
        bottom = np.hstack((Ru, Zero))
        
        S = np.vstack((top, bottom))
        Us, Ss, Vs = np.linalg.svd(S)
        
        e_mach = np.random.uniform(0, 1, k)
        S_plus = Ss[1:k, 1:k] + np.diag(e_mach)
        
        Current = np.hstack((U, Qu))
        U_plus = np.dot(Current, Us)[:, 1:k]

        Current = np.hstack((V, Qv))
        U_plus = np.dot(Current, Vs)[:, 1:k]
        
        ans = np.dot(U_plus, np.dot(Sigma_plus, V_plus.T))
        return ans
        
    
    def LRGeomCG(self):
        
        # Check Initialization!!!!!
        m = self.m
        n = self.n
        k = self.rank
        
        diag = np.random.uniform(0, 1, k)
        diag = np.sort(diag)[::-1]
        
        S0 = np.diag(diag)
        U0 = np.random.uniform(0, 1, (m, k))
        V0 = np.random.uniforn(0, 1, (n, k))
        
        Qu0, Ru0 = np.linalg.qr(U0)
        Qv0, Rv0 = np.linalg.qr(V0)
        
        X_old_svd = [Qu0, S0, Qv0]
        X_new_svd = X_old_svd.copy()
        
        xi_old = np.zeros((m, n))
        xi_old_list = [Qu0, np.zeros((k, k)), Qv0, np.zeros((m, k)), np.zeros((n, k))]
        
        eta_old = xi_old.copy()
        eta_old_list = xi_old_list.copy()
        
        
        while True:
            
            R = Xw - Aw # Calculate them!!!
            
            # Compute the gradient
            (xi_new, xi_new_list) = self.grad(svd, R)
            
            # Check convergence
            if np.linalg.norm(xi_new) <= self.tau:
                break
            
            # Compute a conjugate direction by PR+
            (eta_new, eta_new_list) = conjugate_direction(self, X_old_svd, X_new_svd, xi_old_list, \
                                                                      xi_new_list, eta_old_list)
            
            # Determine an initial step ti from the linearized problem
            t = self.compute_initial_guess(X_new_svd, Rw, eta_new_list)
            
            # Performing Armijo backtracking
            ////////
            

In [27]:
diag = np.random.uniform(0, 1, 4)
diag

array([ 0.55538617,  0.954394  ,  0.03891768,  0.63663842])

In [28]:
diag = np.sort(diag)[::-1]
diag

array([ 0.954394  ,  0.63663842,  0.55538617,  0.03891768])

In [14]:
diag = (np.sort(diag)).flip(axis=0)
diag

AttributeError: 'numpy.ndarray' object has no attribute 'flip'

In [12]:
np.flip(diag, axis = 0)

array([ 0.96371904,  0.38484624,  0.36138523,  0.27141429])